In [1]:
%matplotlib inline
from d2l import tensorflow as d2l
import tensorflow as tf

In [4]:
X = tf.norm(0.0, 1, (1000, 2), tf.float32)
A = d2l.tensor([[1, 2], [-0.1, 0.5]], tf.float32)
b = d2l.tensor([1, 2], tf.float32)
data = d2l.matmul(X, A) + b

AttributeError: module 'tensorflow' has no attribute 'normal'